# 03. Text Classification(Korean)

## 1. 한글 문서의 분류

* 다음무비<http://movie.daum.net>로부터 crawl한 영화리뷰를 이용하여 분류 연습
* 영화리뷰와 영화의 제목을 학습해서 주어진 리뷰내용으로 어떤 영화에 대한 리뷰인지를 예측하고자 함

#### data file 내용
*'신과함께', '코코', '라라랜드', '인피니티 워', '곤지암' 다섯개의 영화에 대해 총 1827개의 리뷰를 수집 csv 파일 안에 리뷰내용, 평점, 영화이름 의 순으로 저장되어 있음

### 1) data 불러오기

In [28]:
import csv

#초기화
text = []    #영화리뷰
y = []     #영화제목

with open('movie_data.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만(NA 제외)
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [29]:
print('Num of samples: {}'.format(len(text)))    #총 리뷰수 보기
print('Movie titles of reivews: {}'.format(set(y)))   #영화제목 보기

Num of samples: 1827
Movie titles of reivews: {'코코', '신과함께', '곤지암', '인피니티 워', '라라랜드'}


### 2) training set, test set으로 분리하기

In [30]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [31]:
len(X_train)   #1872의 0.75

1370

### 3) 형태소 분석기 활용해 형태소 단위로 tokenize

In [32]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import

twitter_tag = Okt()
#twitter_tag = Twitter()

In [33]:
print(twitter_tag.morphs(X_train[1])) #둘째 리뷰에 대해 형태소 단위로 tokenize

['혹시', '나', '하고', '봤는데', '역시', '나다', ';;', '편집', '과', '사운드', '로', '주는', '작은', '공포', '영화', "'", '푸시', "'", '에서', '인상', '깊게', '봤던걸', '여기', '서', '또', '보네', ';;']


In [34]:
twitter_tag.nouns(X_train[1]) #둘째 리뷰에서 명사만 추출

['혹시', '역시', '편집', '사운드', '공포', '영화', '푸시', '인상', '여기', '또']

### 4) 명사만 추출하는 tokenizer 함수를 생성한 뒤 분류기(classifier) 학습

In [35]:
# Twitter 형태소 분석기의 명사만 추출하는 함수를 tokenizer 함수로 사용
def twit_tokenizer(text):
    return twitter_tag.nouns(text)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

## 명사만 추출하는 tokenizer 함수 이용
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
#twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
#하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

## train set, test set을 vector로 변환
X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

## 분류기 학습 및 예측정확도 확인
clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습

print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 1156개의 명사로 이루어짐

Train score 0.8364963503649635
Test score 0.6717724288840262
(1370, 1156)


### 5) test set과 예측내용 확인

In [37]:
X_test[:10] #test data에서 앞 10개를 출력

['졸잼 최고',
 '내용, 음악 , 연기력  무엇하나 빠지는것이 없네요 특히 음악은 계속 찾아 듣게되요^^',
 '아맥2D로 느즈막히 관람.... 히어로가 많이나오지만, 이걸 꽤나 잘 버무려놓음. 뻔한스토리의 틀을 벗어나려 노력한점은 높은점수를 줄만함.... 블럭버스터액션, 영상미는 말이필요없음...... 후속편 기대됨!',
 '후반부터 쫄렸다.',
 '진짜. 솔직히 한국 공포영화중에 이렇게 소재별로인건 정말 오랜만인듯; 지들끼리 소리지르고 정신없이 우왕자왕 심지어 무섭지도않어 효과음만크고 진짜최악임ㅉㅉ',
 '소문난 잔치에 먹을거 없음..ㅜㅜ',
 'good!',
 '아 점수를 줄 수가 없네  화면은 왜그리도 흔들어 데는지........ 재미도 없고 가볍기만하고 .... 최악의 재미없는 배멀미 영화',
 '영화 보면서 펑펑물었네요~ 부모님 사랑에 대해 다시한번 생각하게 했던 영화네요^ ^',
 '슬픈 스토리지만 삶을 돌아보게 하는 영화다. 죄를 지은자는 그 벌을 고스란히 받으리라. 사회 각종범죄자들 뉘우치길 바란다.']

In [38]:
clf.predict(X_test_tfidf[:10]) # test data의 앞 10개에 대한 예측내용

array(['인피니티 워', '라라랜드', '인피니티 워', '곤지암', '곤지암', '인피니티 워', '인피니티 워',
       '신과함께', '코코', '신과함께'], dtype='<U6')

In [39]:
print(y_test[:10]) # test data 앞 10개의 실제 영화제목 #맨 뒤 곤지암, 신과함께 빼고 맞음

['인피니티 워', '라라랜드', '인피니티 워', '곤지암', '곤지암', '인피니티 워', '인피니티 워', '곤지암', '신과함께', '신과함께']


## 2. 성능을 개선하기 위한 노력

### 1) 명사 이외에도 다른 형태소도 모두 포함해 tokenize

In [40]:
# morphs()는 명사를 포함한 모든 형태소를 tokenize
print(twitter_tag.morphs(X_train[1]))

['혹시', '나', '하고', '봤는데', '역시', '나다', ';;', '편집', '과', '사운드', '로', '주는', '작은', '공포', '영화', "'", '푸시', "'", '에서', '인상', '깊게', '봤던걸', '여기', '서', '또', '보네', ';;']


In [41]:
## 명사 대신 모든 형태소를 사용
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)

## vector로 변환
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

## train set 학습 및 성능확인
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)

print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)   #명사만 사용한 것에 비해 train score는 상승, test score는 하락

Train score 0.8963503649635036
Test score 0.649890590809628
(1370, 2260)


### 2) 명사, 동사, 형용사만 tokenize

In [42]:
#pos()는 형태소와 품사를 함께 제공
print(twitter_tag.pos(X_train[1], norm=True, stem=True))

[('혹시', 'Noun'), ('나', 'Josa'), ('하다', 'Verb'), ('보다', 'Verb'), ('역시', 'Noun'), ('나다', 'Verb'), (';;', 'Punctuation'), ('편집', 'Noun'), ('과', 'Josa'), ('사운드', 'Noun'), ('로', 'Josa'), ('주다', 'Verb'), ('작다', 'Adjective'), ('공포', 'Noun'), ('영화', 'Noun'), ("'", 'Punctuation'), ('푸시', 'Noun'), ("'", 'Punctuation'), ('에서', 'Josa'), ('인상', 'Noun'), ('깊다', 'Adjective'), ('보다', 'Verb'), ('여기', 'Noun'), ('서', 'Josa'), ('또', 'Noun'), ('보다', 'Verb'), (';;', 'Punctuation')]


In [43]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
            #result.append('/'.join([word, tag]))  -> 뒤에서는 이걸로해서 단어의 품사를 구분할 수 있도록!
    return result

In [44]:
print(twit_tokenizer2(X_train[1])) # 사용 예

['혹시', '하다', '보다', '역시', '나다', '편집', '사운드', '주다', '작다', '공포', '영화', '푸시', '인상', '깊다', '보다', '여기', '또', '보다']


In [45]:
## 명사, 동사, 형용사를 이용하여 tfidf 생성
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)

## vector로 변경
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

## 학습 및 성능체크
clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 test score가 가장 뛰어남

Train score 0.8715328467153285
Test score 0.6849015317286652
(1370, 1584)


### 3) 모든 형태소를 다 사용해 tokenize + 품사 구분할 수 있도록

In [46]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [47]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#성능이 오히려 떨어지고 품사 표시 없이 전체를 다 사용한 경우에 비해 train은 떨어지고, test는 올라감

Train score 0.8927007299270073
Test score 0.6739606126914661
(1370, 2023)


### 4) Ridge regression(릿지 회귀)

#### Logistic regression 회귀 분석
* 종속 변수와 독립 변수간의 관계를 구체적인 함수로 나타내어 향후 **예측 모델에 사용**
* 종속 변수가 범주형 데이터를 대상으로 하며, 입력 데이터가 주어졌을 때 해당 **데이터의 결과가 특정 분류로 나뉘기때문에 일종의 분류 (classification) 기법에 해당**
* 하지만 텍스트 마이닝에서는 추정해야 할 계수가 vector의 크기(단어의 수)만큼 존재하므로, 과적합이 발생하기 쉽고 많은 데이터 셋이 필요 
* 그럼에도 불구하고 잘 작동하는 편이며 정규화(regulation)을 이용해 과적합 해결 노력

#### Ridge regression
* 목적 함수에 추정할 계수(parameter)에 대한 **L2 norm(규제항)을 추가하여 모형의 과적합을 방지

In [48]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 올라가는 현상이 벌어짐
# test score가 올라갔으나 명사, 형용사, 동사를 쓴 것보다 떨어짐

Train set score: 0.944
Test set score: 0.676


### 5) Lasso regression(라쏘 회귀)

#### Lasso regression
* L1 norm을 규제항으로 사용함으로써 0에 가까운 계수를 0으로 만들어 **영향을 거의 미치지 않는 단어들을 제외**

In [49]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])
# 현재까지 방법 중에 train score, test score 모두 가장 떨어짐

Train set score: 0.718
Test set score: 0.643
Used features count: 240 out of 2023


### 6) LSA(Latent Semantic Analysis, 잠재의미분석)
* a technique in natural language processing, in particular distributional semantics, of *analyzing relationships between a set of documents and the terms they contain* by **producing a set of concepts related to the documents and terms**. LSA assumes that **words that are close in meaning will occur in similar pieces of text** (the distributional hypothesis, 분배/분해 가설)

* **SVD**(Singular Vector Decomposition), 특이값 분해 : 주어진 dtm(document term matrix, A)을 A=U\*Σ\*V^T의 형태로 분해
* **Truncated SVD**, 잠재의미분석 : Σ 행렬의 대각원소(특이값) 가운데 잠재의미군 비중에 큰 상위 t개만 골라내어 A'=Ut(m, t)\*Σ(t, t)\*VtT(t, n)의 형태로 분해되어 A에 근사한 A'값을 구할 수 있음

In [50]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_)   #특이값
print(svd.components_.shape)

[0.01102934 0.01405529 0.01144551 0.0110093  0.00918016 0.00876149
 0.00836851 0.00795922 0.00777003 0.00737423 0.00707199 0.00665546
 0.00659729 0.00616382 0.00588976 0.00563969 0.00546929 0.00543044
 0.00524396 0.00523751 0.0051354  0.00507564 0.00494917 0.00478661
 0.00466253 0.0046021  0.00455272 0.00452943 0.00432636 0.0043043
 0.00425346 0.00416261 0.00410043 0.00401296 0.00399825 0.00396182
 0.00392779 0.00385998 0.00373325 0.00369469 0.00365484 0.00363113
 0.00357832 0.00354544 0.00349848 0.00346928 0.00337552 0.00334918
 0.00332458 0.00330956 0.00323571 0.00321667 0.00315217 0.00314053
 0.00309441 0.00307477 0.0030649  0.00301359 0.00300256 0.00296081
 0.00293836 0.00291433 0.00288938 0.00288701 0.00286437 0.00281526
 0.00280011 0.00276482 0.00274032 0.00271676 0.00269105 0.00266254
 0.00264105 0.00262961 0.00260318 0.00258836 0.00257263 0.0025549
 0.00253862 0.00252722 0.00251281 0.00248844 0.00248194 0.00245121
 0.00244755 0.00241116 0.00239864 0.0023803  0.00236743 0.002359

In [51]:
## 특이값 분해(SVD, singular vector decomposition)로 차원 축소
X_train_svd = svd.transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)
#선택된 component를 이용하여 2,000개의 feature로부터 feature extract (차원축소=dimension reduce)

## 학습 및 성능체크
from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))
# 마찬가지로 train, test score 모두 lasso 회귀 다음으로 가장 떨어짐

Train set score: 0.771
Test set score: 0.654


### 7) Naïve Bayes(NB)

* Wikipedia: a popular (baseline) method for text categorization, the problem of judging documents as belonging to one category or the other (such as spam or legitimate, sports or politics, etc.) with word frequencies (count vector) as the features.

* (x1, …, xn) 의 단어집합으로 이루어진 문서가 분류 Ck에 속할 확률
* 예제 <http://bcho.tistory.com/m/1010>

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

## 명사, 동사, 형용사만 tokenize
cv = CountVectorizer(tokenizer=twit_tokenizer2, min_df=2).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

## 학습 및 성능체크
from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))
# train score는 명사, 동사, 형용사를 사용한 경우보다 다소 떨어짐
# test score는 사용한 방법들 중 가장 높게 나옴!

C:\Users\Okyoung\Downloads\ANACONDA\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Train set dimension: (1370, 1584)
Test set dimension: (457, 1584)
Train set score: 0.801
Test set score: 0.685


#### 사용한 방법들의 train, test set score 비교하기

In [54]:
import pandas as pd
scores = pd.read_csv('C:/Users/Okyoung/Desktop/Business Analytics_캡스톤디자인/BusinessAnalytics/HW/compare_score.csv')

scores   #나이브 베이즈(NB)가 가장 좋은 test score를 보유

,method,train,test
0,명사,0.8364,0.6717
1,형태소,0.8963,0.6498
2,명동형,0.8715,0.6849
3,형태소+품사,0.8927,0.6739
4,릿지,0.9440,0.6760
5,라쏘,0.7180,0.6430
6,LSA,0.7710,0.6540
7,나이브 베이즈,0.8010,0.6850
